In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from CV.preprocessing.ImageToArrayPreprocessor import ImageToArrayPreprocessor
from CV.preprocessing.aspectawarepreprocessor import AspectAwarePreprocessor
from CV.datasets.SimpleDatasetLoader import SimpleDatasetLoader
from CV.nn.conv.fcheadnet import FCHeadNet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from imutils import paths
import numpy as np
import argparse
import os
from keras.applications import ResNet50
#我再
'''aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")'''

# grab the list of images that we’ll be describing, then extract
# the class label names from the image paths
print("[INFO] loading images...")
imagePaths = list(paths.list_images("data3"))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

# initialize the image preprocessors
aap = AspectAwarePreprocessor(128,128)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixel intensities to
# the range [0, 1]

sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose = 500)
data = data.astype("float")/255.0

(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.1, random_state=42)

#convert the labels from integers to vectors
#trainY = LabelBinarizer().fit_transform(trainY)
#testY = LabelBinarizer().fit_transform(testY)

from keras.utils import to_categorical
lb = LabelBinarizer()
#train_y = lb.fit_transform(train_y)
#test_y = lb.fit_transform(test_y)
lb.fit(trainY)
trainY=lb.transform(trainY)
testY = lb.transform(testY)
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# load the VGG16 network, ensuring the head FC layer sets are left
# off

#baseModel = VGG16(weights="imagenet", include_top=False,
#                 input_tensor=Input(shape=(128,128,3)))
baseModel = ResNet50(weights="imagenet", include_top=False,
                     input_tensor=Input(shape=(128,128,3)))

# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames),256)

# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, output = headModel)

# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False
    
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy",optimizer=opt,
             metrics=["accuracy"])

# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
'''model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
    validation_data=(testX, testY), epochs=25,
    steps_per_epoch=len(trainX) // 32, verbose=1)'''
model.fit(trainX, trainY, batch_size=32,
    validation_data=(testX, testY), epochs=25,
     verbose=1)
# evaluate the network after initialization
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                           predictions.argmax(axis=1),target_names=classNames))


# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True
# 用小lr的sgd重新finetune
print("[INFO] re-compile model...")
opt = SGD(lr = 0.001)
model.compile(loss="categorical_crossentropy", optimizer = opt,
             metrics=["accuracy"])

print("[INFO] fine-tuning model...")
model.fit(trainX, trainY, batch_size=32,
    validation_data=(testX, testY), epochs=25,
     verbose=1)

# evaluate the network after fine-tune
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                           predictions.argmax(axis=1),target_names=classNames))

# save the model to disk
print("[INFO] serializing model...")
model.save("flowers17.model")

[INFO] loading images...
[INFO] processed 500/4726
[INFO] processed 1000/4726
[INFO] processed 1500/4726
[INFO] processed 2000/4726
[INFO] processed 2500/4726
[INFO] processed 3000/4726
[INFO] processed 3500/4726
[INFO] processed 4000/4726
[INFO] processed 4500/4726


ValueError: Negative dimension size caused by subtracting 7 from 4 for 'avg_pool_1/AvgPool' (op: 'AvgPool') with input shapes: [?,4,4,2048].

In [5]:
baseModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation